In [3]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau
from sklearn.model_selection import train_test_split
import numpy as np

# Precomputed Lookup Table for log2 values
LOG2_LOOKUP_TABLE = {i: np.log2(i + 1e-8) for i in range(1, 256)}  # For values 1-255

def lookup_log2(x):
    """
    Approximate log2 using a precomputed lookup table for values in [1, 255].
    """
    x_clipped = tf.clip_by_value(x, 1, 255)  # Clip values to [1, 255]
    x_int = tf.cast(x_clipped, tf.int32)
    lookup_tensor = tf.constant([LOG2_LOOKUP_TABLE[i] for i in range(1, 256)], dtype=tf.float32)
    log2_approx = tf.gather(lookup_tensor, x_int - 1)  # Subtract 1 since index starts at 0
    return log2_approx

# Taylor series expansion for log(1 + x)
def taylor_log1p(x, terms=5):
    """
    Compute log(1 + x) using Taylor series expansion.
    """
    result = tf.zeros_like(x)
    for n in range(1, terms + 1):
        term = tf.pow(-1.0, n + 1) * tf.pow(x, n) / n
        result += term
    return result

# Quantization function using Taylor approximation and lookup
def log2_quantize(x, method="floor", fractional_bits=3):
    """
    Quantize log2(x) using lookup tables and Taylor series.
    """
    log2_x = lookup_log2(x)
    small_x = x - tf.ones_like(x)  # Adjust for values close to 1
    refined_log = taylor_log1p(small_x, terms=5)
    log2_x = tf.where(x < 2.0, refined_log, log2_x)

    if method == "floor":
        return tf.floor(log2_x)
    elif method == "round":
        integer_part = tf.floor(log2_x)
        fractional_part = log2_x - integer_part
        threshold = (2**fractional_bits - 1) / (2**fractional_bits)
        return integer_part + tf.cast(fractional_part >= threshold, tf.float32)
    else:
        raise ValueError("Invalid quantization method. Choose 'floor' or 'round'.")

# Custom CNN with Your Desired Architecture and Log Quantization
class LogCNN(tf.keras.Model):
    def __init__(self, input_shape, num_classes):
        super(LogCNN, self).__init__()
        self.conv1a = layers.Conv2D(64, (3, 3), padding='same')
        self.bn1a = layers.BatchNormalization()
        self.conv1b = layers.Conv2D(64, (3, 3), padding='same')
        self.pool1 = layers.MaxPooling2D((2, 2))
        self.dropout1 = layers.Dropout(0.2)
        
        self.conv2a = layers.Conv2D(128, (3, 3), padding='same')
        self.bn2a = layers.BatchNormalization()
        self.conv2b = layers.Conv2D(128, (3, 3), padding='same')
        self.pool2 = layers.MaxPooling2D((2, 2))
        self.dropout2 = layers.Dropout(0.3)

        self.conv3a = layers.Conv2D(256, (3, 3), padding='same')
        self.bn3a = layers.BatchNormalization()
        self.conv3b = layers.Conv2D(256, (3, 3), padding='same')
        self.pool3 = layers.MaxPooling2D((2, 2))
        self.dropout3 = layers.Dropout(0.4)

        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512)
        self.bn4 = layers.BatchNormalization()
        self.dropout4 = layers.Dropout(0.4)
        self.fc2 = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        # Quantize inputs
        x = log2_quantize(inputs, method="floor")
        
        # First Convolutional Block
        x = self.conv1a(x)
        x = tf.nn.relu(x)
        x = self.bn1a(x)
        x = self.conv1b(x)
        x = tf.nn.relu(x)
        x = self.pool1(x)
        x = self.dropout1(x)

        # Second Convolutional Block
        x = self.conv2a(x)
        x = tf.nn.relu(x)
        x = self.bn2a(x)
        x = self.conv2b(x)
        x = tf.nn.relu(x)
        x = self.pool2(x)
        x = self.dropout2(x)

        # Third Convolutional Block
        x = self.conv3a(x)
        x = tf.nn.relu(x)
        x = self.bn3a(x)
        x = self.conv3b(x)
        x = tf.nn.relu(x)
        x = self.pool3(x)
        x = self.dropout3(x)

        # Fully Connected Layers
        x = self.flatten(x)
        x = self.fc1(x)
        x = tf.nn.relu(x)
        x = self.bn4(x)
        x = self.dropout4(x)
        return self.fc2(x)

# Main Execution
if __name__ == "__main__":
    # Load MNIST dataset
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    # Preprocess the data
    x_train = x_train.reshape((-1, 28, 28, 1)).astype("float32") / 255.0
    x_test = x_test.reshape((-1, 28, 28, 1)).astype("float32") / 255.0

    # One-hot encode the labels
    y_train = to_categorical(y_train, 10)
    y_test = to_categorical(y_test, 10)

    # Split data into training and validation sets
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

    # Data augmentation for training data
    train_datagen = ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1,
        shear_range=0.1
    )

    # No augmentation for validation data
    val_datagen = ImageDataGenerator()

    # Flow generators
    train_generator = train_datagen.flow(x_train, y_train, batch_size=128)
    val_generator = val_datagen.flow(x_val, y_val, batch_size=128)

    # Define callbacks
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)

    # Define input shape and number of classes
    input_shape = (28, 28, 1)
    num_classes = 10

    # Instantiate and compile the model
    model = LogCNN(input_shape, num_classes)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

    # Train the model
    model.fit(train_generator, 
              epochs=10, 
              validation_data=val_generator, 
              callbacks=[reduce_lr])

    # Evaluate the model
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/10


C:\Users\prabh\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


422/422 ━━━━━━━━━━━━━━━━━━━━ 111s 255ms/step - accuracy: 0.6826 - loss: 0.8668 - val_accuracy: 0.9805 - val_loss: 0.0561 - learning_rate: 0.0010
Epoch 2/10
107/422 ━━━━━━━━━━━━━━━━━━━━ 1:28 282ms/step - accuracy: 0.9715 - loss: 0.0939

KeyboardInterrupt: 

In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np

lookup_min = 0.01
fixed_point_scale = 1 << 32

# Precompute lookup tables for simplified Taylor approximation
def create_lookup_tables():
    mantissa_values = np.linspace(lookup_min, 256.0, 1024)
    exponent_values = np.floor(np.log2(mantissa_values))
    mantissa_normalized = mantissa_values / (2 ** exponent_values) - 1
    log2_mantissa_approx = mantissa_normalized - (mantissa_normalized ** 2) / 2  # Simplified Taylor series
    return (tf.constant(log2_mantissa_approx, dtype=tf.float32), 
            tf.constant(exponent_values, dtype=tf.float32))

log2_mantissa_table, exponent_table = create_lookup_tables()

def taylor_approx(x):
    x_clipped = tf.clip_by_value(x, lookup_min, 256.0)
    indices = tf.cast((x_clipped - lookup_min) / (256.0 - lookup_min) * 1023, tf.int32)
    log2_mantissa = tf.gather(log2_mantissa_table, indices)
    exponent = tf.gather(exponent_table, indices)
    return log2_mantissa + exponent

def log_quantize(x):
    log2_x = taylor_approx(x)
    return tf.floor(log2_x)

class LogCNN(tf.keras.Model):
    def __init__(self, input_shape, num_classes):
        super(LogCNN, self).__init__()
        self.conv1 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(64, (3, 3), padding='same', activation='relu')
        self.pool1 = layers.MaxPooling2D((2, 2))
        self.dropout1 = layers.Dropout(0.25)
        self.conv3 = layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.bn2 = layers.BatchNormalization()
        self.conv4 = layers.Conv2D(128, (3, 3), padding='same', activation='relu')
        self.pool2 = layers.MaxPooling2D((2, 2))
        self.dropout2 = layers.Dropout(0.25)
        self.conv5 = layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.bn3 = layers.BatchNormalization()
        self.conv6 = layers.Conv2D(256, (3, 3), padding='same', activation='relu')
        self.pool3 = layers.MaxPooling2D((2, 2))
        self.dropout3 = layers.Dropout(0.25)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512, activation='relu')
        self.bn4 = layers.BatchNormalization()
        self.dropout4 = layers.Dropout(0.5)
        self.fc2 = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = log_quantize(inputs)
        x = self.conv1(x)
        x = self.bn1(x)
        x = log_quantize(x)
        x = self.conv2(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        x = log_quantize(x)
        x = self.conv3(x)
        x = self.bn2(x)
        x = log_quantize(x)
        x = self.conv4(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        x = log_quantize(x)
        x = self.conv5(x)
        x = self.bn3(x)
        x = log_quantize(x)
        x = self.conv6(x)
        x = self.pool3(x)
        x = self.dropout3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn4(x)
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

if __name__ == "__main__":
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    x_train = tf.image.resize(x_train, (32, 32))
    x_test = tf.image.resize(x_test, (32, 32))
    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)
    input_shape = (32, 32, 1)
    num_classes = 10
    model = LogCNN(input_shape, num_classes)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(x_train, y_train, batch_size=64, epochs=10, validation_split=0.1)
    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/10


C:\Users\prabh\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:735: UserWarning: Gradients do not exist for variables ['log_cnn/conv2d/kernel', 'log_cnn/conv2d/bias', 'log_cnn/batch_normalization/gamma', 'log_cnn/batch_normalization/beta', 'log_cnn/conv2d_1/kernel', 'log_cnn/conv2d_1/bias', 'log_cnn/conv2d_2/kernel', 'log_cnn/conv2d_2/bias', 'log_cnn/batch_normalization_1/gamma', 'log_cnn/batch_normalization_1/beta', 'log_cnn/conv2d_3/kernel', 'log_cnn/conv2d_3/bias', 'log_cnn/conv2d_4/kernel', 'log_cnn/conv2d_4/bias', 'log_cnn/batch_normalization_2/gamma', 'log_cnn/batch_normalization_2/beta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


844/844 ━━━━━━━━━━━━━━━━━━━━ 119s 136ms/step - accuracy: 0.5972 - loss: 3.3347 - val_accuracy: 0.8937 - val_loss: 0.3289
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 115s 136ms/step - accuracy: 0.8923 - loss: 0.3163 - val_accuracy: 0.9280 - val_loss: 0.2366
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 117s 139ms/step - accuracy: 0.9139 - loss: 0.2543 - val_accuracy: 0.9033 - val_loss: 0.2960
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 114s 135ms/step - accuracy: 0.9221 - loss: 0.2303 - val_accuracy: 0.9330 - val_loss: 0.2154
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 114s 135ms/step - accuracy: 0.9317 - loss: 0.2064 - val_accuracy: 0.9357 - val_loss: 0.2060
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 87s 103ms/step - accuracy: 0.9312 - loss: 0.2002 - val_accuracy: 0.9352 - val_loss: 0.2067
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 72s 85ms/step - accuracy: 0.9371 - loss: 0.1890 - val_accuracy: 0.9355 - val_loss: 0.2032
Epoch 8/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 71s 84ms/step - accuracy: 0.9380 - loss: 0.1842 - 

KeyboardInterrupt: 

In [11]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
from sklearn.model_selection import train_test_split

# Constants
lookup_min = 0.01
lookup_max = 256.0
lookup_size = 1024

# Precompute lookup table for log2 values
def create_lookup_table():
    """
    Create a lookup table for log2(x) values.
    The table has precomputed log2 values for inputs in the range [lookup_min, lookup_max].
    """
    lookup_values = np.log2(np.linspace(lookup_min, lookup_max, lookup_size))
    return tf.constant(lookup_values, dtype=tf.float32)

lookup_table = create_lookup_table()

# Lookup table-based approximation for log2(x)
def log2_lookup(x):
    """
    Approximate log2(x) using a lookup table.
    Parameters:
        x: Tensor of input values.
    Returns:
        Tensor of approximated log2 values.
    """
    x_clipped = tf.clip_by_value(x, lookup_min, lookup_max)  # Ensure input is in valid range
    indices = tf.cast((x_clipped - lookup_min) / (lookup_max - lookup_min) * (lookup_size - 1), tf.int32)
    return tf.gather(lookup_table, indices)

# Logarithmic Quantization
def log_quantize(x):
    log2_x = log2_lookup(x)  # Use lookup table for log2 approximation
    return tf.floor(log2_x)

class LogCNN(tf.keras.Model):
    def __init__(self, input_shape, num_classes):
        super(LogCNN, self).__init__()
        self.conv1 = layers.Conv2D(64, (3, 3), padding='same')
        self.act1 = layers.LeakyReLU(alpha=0.1)
        self.bn1 = layers.BatchNormalization()
        self.conv2 = layers.Conv2D(64, (3, 3), padding='same')
        self.act2 = layers.LeakyReLU(alpha=0.1)
        self.pool1 = layers.AveragePooling2D((2, 2))
        self.dropout1 = layers.Dropout(0.1)
        self.conv3 = layers.Conv2D(128, (3, 3), padding='same')
        self.act3 = layers.LeakyReLU(alpha=0.1)
        self.bn2 = layers.BatchNormalization()
        self.conv4 = layers.Conv2D(128, (3, 3), padding='same')
        self.act4 = layers.LeakyReLU(alpha=0.1)
        self.pool2 = layers.AveragePooling2D((2, 2))
        self.dropout2 = layers.Dropout(0.2)
        self.conv5 = layers.Conv2D(256, (3, 3), padding='same')
        self.act5 = layers.LeakyReLU(alpha=0.1)
        self.bn3 = layers.BatchNormalization()
        self.conv6 = layers.Conv2D(256, (3, 3), padding='same')
        self.act6 = layers.LeakyReLU(alpha=0.1)
        self.pool3 = layers.AveragePooling2D((2, 2))
        self.dropout3 = layers.Dropout(0.2)
        self.flatten = layers.Flatten()
        self.fc1 = layers.Dense(512, activation='relu')
        self.bn4 = layers.BatchNormalization()
        self.dropout4 = layers.Dropout(0.3)
        self.fc2 = layers.Dense(num_classes, activation='softmax')

    def call(self, inputs):
        x = log_quantize(inputs)
        x = self.conv1(x)
        x = self.act1(x)
        x = self.bn1(x)
        x = log_quantize(x)
        x = self.conv2(x)
        x = self.act2(x)
        x = self.pool1(x)
        x = self.dropout1(x)
        x = log_quantize(x)
        x = self.conv3(x)
        x = self.act3(x)
        x = self.bn2(x)
        x = log_quantize(x)
        x = self.conv4(x)
        x = self.act4(x)
        x = self.pool2(x)
        x = self.dropout2(x)
        x = log_quantize(x)
        x = self.conv5(x)
        x = self.act5(x)
        x = self.bn3(x)
        x = log_quantize(x)
        x = self.conv6(x)
        x = self.act6(x)
        x = self.pool3(x)
        x = self.dropout3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.bn4(x)
        x = self.dropout4(x)
        x = self.fc2(x)
        return x

if __name__ == "__main__":
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    x_train = np.expand_dims(x_train, -1)
    x_test = np.expand_dims(x_test, -1)
    x_train = tf.image.resize(x_train, (32, 32))
    x_test = tf.image.resize(x_test, (32, 32))
    mean, std = np.mean(x_train), np.std(x_train)
    x_train = (x_train - mean) / std
    x_test = (x_test - mean) / std

    y_train = tf.keras.utils.to_categorical(y_train, 10)
    y_test = tf.keras.utils.to_categorical(y_test, 10)

    # Convert to NumPy arrays before splitting
    x_train_np = x_train.numpy()
    y_train_np = y_train  # Already a NumPy array from to_categorical

    # Manually split the training data
    x_train, x_val, y_train, y_val = train_test_split(x_train_np, y_train_np, test_size=0.1, random_state=42)

    # Convert back to tensors
    x_train = tf.convert_to_tensor(x_train)
    y_train = tf.convert_to_tensor(y_train)
    x_val = tf.convert_to_tensor(x_val)
    y_val = tf.convert_to_tensor(y_val)

    datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rotation_range=10,
        width_shift_range=0.1,
        height_shift_range=0.1,
        zoom_range=0.1
    )
    datagen.fit(x_train)

    input_shape = (32, 32, 1)
    num_classes = 10
    model = LogCNN(input_shape, num_classes)

    lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=0.001,
        decay_steps=10000,
        decay_rate=0.9
    )
    optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule, clipnorm=1.0)

    model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy"])
    model.fit(datagen.flow(x_train, y_train, batch_size=64), epochs=10, validation_data=(x_val, y_val))

    test_loss, test_acc = model.evaluate(x_test, y_test, verbose=2)
    print(f"Test Accuracy: {test_acc:.4f}")


Epoch 1/10


C:\Users\prabh\anaconda3\Lib\site-packages\keras\src\optimizers\base_optimizer.py:735: UserWarning: Gradients do not exist for variables ['log_cnn_5/conv2d_30/kernel', 'log_cnn_5/conv2d_30/bias', 'log_cnn_5/batch_normalization_20/gamma', 'log_cnn_5/batch_normalization_20/beta', 'log_cnn_5/conv2d_31/kernel', 'log_cnn_5/conv2d_31/bias', 'log_cnn_5/conv2d_32/kernel', 'log_cnn_5/conv2d_32/bias', 'log_cnn_5/batch_normalization_21/gamma', 'log_cnn_5/batch_normalization_21/beta', 'log_cnn_5/conv2d_33/kernel', 'log_cnn_5/conv2d_33/bias', 'log_cnn_5/conv2d_34/kernel', 'log_cnn_5/conv2d_34/bias', 'log_cnn_5/batch_normalization_22/gamma', 'log_cnn_5/batch_normalization_22/beta'] when minimizing the loss. If using `model.compile()`, did you forget to provide a `loss` argument?
  warnings.warn(


844/844 ━━━━━━━━━━━━━━━━━━━━ 72s 82ms/step - accuracy: 0.4717 - loss: 4.3616 - val_accuracy: 0.9002 - val_loss: 0.3284
Epoch 2/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 68s 81ms/step - accuracy: 0.8592 - loss: 0.4419 - val_accuracy: 0.9220 - val_loss: 0.2371
Epoch 3/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 70s 82ms/step - accuracy: 0.8906 - loss: 0.3432 - val_accuracy: 0.9240 - val_loss: 0.2120
Epoch 4/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 67s 80ms/step - accuracy: 0.9055 - loss: 0.2975 - val_accuracy: 0.9413 - val_loss: 0.1816
Epoch 5/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 70s 83ms/step - accuracy: 0.9168 - loss: 0.2595 - val_accuracy: 0.9630 - val_loss: 0.1176
Epoch 6/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 69s 82ms/step - accuracy: 0.9248 - loss: 0.2377 - val_accuracy: 0.9538 - val_loss: 0.1400
Epoch 7/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 70s 83ms/step - accuracy: 0.9289 - loss: 0.2241 - val_accuracy: 0.9545 - val_loss: 0.1401
Epoch 8/10
844/844 ━━━━━━━━━━━━━━━━━━━━ 72s 86ms/step - accuracy: 0.9283 - loss: 0.2251 - val_accurac